In [5]:
def unique(list1):
 
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    # for x in unique_list:
    #     print x,
    return(unique_list)

In [6]:
from datasets import load_dataset
dataset = load_dataset("gretelai/symptom_to_diagnosis")

train = dataset["train"]
train_label = train[:]["output_text"]

In [8]:
labels=(unique(train_label))
labels_dict = {labels[i]: i for i in range(len(labels))}
print(labels_dict)

{'psoriasis': 0, 'urinary tract infection': 1, 'fungal infection': 2, 'allergy': 3, 'peptic ulcer disease': 4, 'gastroesophageal reflux disease': 5, 'varicose veins': 6, 'hypertension': 7, 'impetigo': 8, 'chicken pox': 9, 'common cold': 10, 'cervical spondylosis': 11, 'bronchial asthma': 12, 'typhoid': 13, 'drug reaction': 14, 'jaundice': 15, 'dengue': 16, 'diabetes': 17, 'pneumonia': 18, 'arthritis': 19, 'malaria': 20, 'migraine': 21}


In [10]:
batch_size = 16 # can experiment with different sizes
dataset = dataset.rename_column("output_text", "labels")
dataset = dataset.class_encode_column("labels")
dataset = dataset.rename_column("input_text", "text")
# dataset = dataset.map(input_columns="output_text", remove_columns="output_text", batched=True, batch_size=batch_size)
print(dataset)

Casting to class labels:   0%|          | 0/853 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/212 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 853
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 212
    })
})


In [11]:
print(dataset["train"].features)

{'labels': ClassLabel(names=['allergy', 'arthritis', 'bronchial asthma', 'cervical spondylosis', 'chicken pox', 'common cold', 'dengue', 'diabetes', 'drug reaction', 'fungal infection', 'gastroesophageal reflux disease', 'hypertension', 'impetigo', 'jaundice', 'malaria', 'migraine', 'peptic ulcer disease', 'pneumonia', 'psoriasis', 'typhoid', 'urinary tract infection', 'varicose veins'], id=None), 'text': Value(dtype='string', id=None)}


In [12]:
train = dataset["train"]
# train_label = train[:]["label"]
# print(train_label)

# Split the 10% test + valid in half test, half valid
# test_valid = dataset["test"].train_test_split(0.5)

# train_test_valid_dataset = DatasetDict({
#     'train': train,
#     'test': test_valid['test'],
#     'validation': test_valid['train']})

train_test_valid_dataset = dataset

print(train_test_valid_dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 853
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 212
    })
})


In [16]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)

def preprocess(sample):
    return tokenizer(sample["text"], truncation=True)

In [17]:
tokenized_dataset = train_test_valid_dataset.map(preprocess, batched=True, batch_size=batch_size)
print(tokenized_dataset)


Map:   0%|          | 0/853 [00:00<?, ? examples/s]

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 853
    })
    test: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 212
    })
})


In [18]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# DataCollatorForTokenClassification

2024-05-02 19:46:17.806204: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 19:46:19.150348: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [19]:
from evaluate import load
metric = load('accuracy')

In [20]:
import numpy as np

def compute_metrics(preds):
    logits, labels = preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [27]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# batch_size = 110 # can experiment with different sizes

args = TrainingArguments(
    "test", # directory to save the model
    evaluation_strategy = "epoch", # evaluate after each epoch
    save_strategy = "epoch", # save after each epoch
    learning_rate=2e-5, # the learning rate to use
    per_device_train_batch_size=batch_size, # the batch size
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20, # number of epochs; 5 took about 30 minutes
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy" # metric associated with COLA GLUE
)

In [28]:
# autoload a model from the base for sequence classification,
# we pass 22 labels
label2id = labels_dict
id2label = {v: k for k, v in labels_dict.items()}
print(id2label)
print(label2id)

{0: 'psoriasis', 1: 'urinary tract infection', 2: 'fungal infection', 3: 'allergy', 4: 'peptic ulcer disease', 5: 'gastroesophageal reflux disease', 6: 'varicose veins', 7: 'hypertension', 8: 'impetigo', 9: 'chicken pox', 10: 'common cold', 11: 'cervical spondylosis', 12: 'bronchial asthma', 13: 'typhoid', 14: 'drug reaction', 15: 'jaundice', 16: 'dengue', 17: 'diabetes', 18: 'pneumonia', 19: 'arthritis', 20: 'malaria', 21: 'migraine'}
{'psoriasis': 0, 'urinary tract infection': 1, 'fungal infection': 2, 'allergy': 3, 'peptic ulcer disease': 4, 'gastroesophageal reflux disease': 5, 'varicose veins': 6, 'hypertension': 7, 'impetigo': 8, 'chicken pox': 9, 'common cold': 10, 'cervical spondylosis': 11, 'bronchial asthma': 12, 'typhoid': 13, 'drug reaction': 14, 'jaundice': 15, 'dengue': 16, 'diabetes': 17, 'pneumonia': 18, 'arthritis': 19, 'malaria': 20, 'migraine': 21}


In [29]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels_dict))

trainer = Trainer(
    model,  # the pre-trained model
    args,  # the TrainingAgruments, defined above
    train_dataset=tokenized_dataset["train"], # the training dataset
    eval_dataset=tokenized_dataset["test"], # the validation dataset
    tokenizer=tokenizer, # our tokenizer
    data_collator=data_collator, # the collator we defined above
    compute_metrics=compute_metrics # our function for computing the metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.674938,0.537736
2,No log,1.983136,0.773585
3,No log,1.452953,0.811321
4,No log,1.053502,0.872642
5,No log,0.771854,0.896226
6,No log,0.551206,0.924528
7,No log,0.406740,0.943396
8,No log,0.312784,0.943396
9,No log,0.242586,0.948113
10,1.156500,0.227410,0.933962


Checkpoint destination directory test/checkpoint-54 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory test/checkpoint-108 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory test/checkpoint-162 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory test/checkpoint-216 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory test/checkpoint-270 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1080, training_loss=0.559446347642828, metrics={'train_runtime': 876.7493, 'train_samples_per_second': 19.458, 'train_steps_per_second': 1.232, 'total_flos': 260102697721620.0, 'train_loss': 0.559446347642828, 'epoch': 20.0})

In [34]:
# Split the 10% test + valid in half test, half valid
dataset2_train = tokenized_dataset["train"].train_test_split(0.5)
dataset2_test = tokenized_dataset["test"]

dataset2 = DatasetDict({
    'train': dataset2_train["train"],
    'test': dataset2_test})

In [48]:
args2 = TrainingArguments(
    "model_2", # directory to save the model
    evaluation_strategy = "epoch", # evaluate after each epoch
    save_strategy = "epoch", # save after each epoch
    learning_rate=2e-5, # the learning rate to use
    per_device_train_batch_size=batch_size, # the batch size
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20, # number of epochs; 5 took about 30 minutes
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy" # metric associated with COLA GLUE
)

model2 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels_dict))

trainer2 = Trainer(
    model2,  # the pre-trained model
    args2,  # the TrainingAgruments, defined above
    train_dataset=dataset2["train"], # the training dataset
    eval_dataset=dataset2["test"], # the validation dataset
    tokenizer=tokenizer, # our tokenizer
    data_collator=data_collator, # the collator we defined above
    compute_metrics=compute_metrics # our function for computing the metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
trainer2.train()

Epoch,Training Loss,Validation Loss


ValueError: I/O operation on closed file